# TorQ: Create Everything
This notebook will use the AWS boto3 APIs to create the needed resources for a TorQ based application. The notebook will first clone the relevant gihub code (TorQ and TorQ AMazon FinSpace Starter Pack) then proceed to create the necessary AWS resources. 

Once you have create all clusters, you can see how to query for data through the gateway, see the [pykx_query_all](pykx_query_all.ipynb) notebook

To cleanup (delete) all resources, run the [delete_all](delete_all.ipynb) notebook.

## AWS Resources Created
- Database   
- Changeset to add data to database   
- Scaling Group that will contain all clusters   
- Shared Volume   
- Dataview of database on the shared volume   
- Clusters

This notebook is based on the TorQ Amazon FinSpace starter pack but uses Scaling Groups and Shared Volumes for cost savings.

[TorQ Amazon FinSpace Starter Pack](https://dataintellecttech.github.io/TorQ-Amazon-FinSpace-Starter-Pack/)



In [1]:
!rm torq_app.zip

rm: cannot remove ‘torq_app.zip’: No such file or directory


In [2]:
!rm -rf TorQ TorQ-Amazon-FinSpace-Starter-Pack

In [3]:
!git -c advice.detachedHead=false clone --depth 1 --branch v5.0.2 https://github.com/DataIntellectTech/TorQ.git 

Cloning into 'TorQ'...
remote: Enumerating objects: 793, done.
remote: Counting objects: 100% (793/793), done.
remote: Compressing objects: 100% (546/546), done.
remote: Total 793 (delta 195), reused 595 (delta 174), pack-reused 0
Receiving objects: 100% (793/793), 5.62 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (195/195), done.


In [4]:
!git -c advice.detachedHead=false clone --depth 1 --branch  v1.0.1 https://github.com/DataIntellectTech/TorQ-Amazon-FinSpace-Starter-Pack.git 

Cloning into 'TorQ-Amazon-FinSpace-Starter-Pack'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 158 (delta 41), reused 118 (delta 36), pack-reused 0
Receiving objects: 100% (158/158), 32.94 MiB | 41.54 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [5]:
!ln -s ../finspace_torq.q TorQ-Amazon-FinSpace-Starter-Pack

In [6]:
import os
import subprocess
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env import *

from clusters import *

# ----------------------------------------------------------------

TORQ_CODEBASE="TorQ"
TORQ_FINSPACE_CODEBASE="TorQ-Amazon-FinSpace-Starter-Pack"

SCALING_GROUP_NAME="SCALING_GROUP_torq"

VOLUME_NAME="SHARED_torq"

# Managed KX Databases
DB_NAME="finspace-database"

DBVIEW_NAME=f"{DB_NAME}_DBVIEW"

# Source data directory
SOURCE_DATA_DIR=f"{TORQ_FINSPACE_CODEBASE}/hdb"

# Code directory
CODEBASE="torq_app"

# S3 Destinations
S3_CODE_PATH="code"
S3_DATA_PATH="data"

NODE_TYPE="kx.sg.4xlarge"

DATABASE_CONFIG=[{ 
    'databaseName': DB_NAME,
    'dataviewName': DBVIEW_NAME
    }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

NAS1_CONFIG= {
        'type': 'SSD_250',
        'size': 1200
}

In [7]:
# triggers credential get
session=None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Create the Database
Create a database from the supplied data in hdb.tar.gz.  

## Stage HDB Data on S3
Using AWS cli, copy hdb to staging bucket

In [8]:
S3_DEST=f"s3://{S3_BUCKET}/{S3_DATA_PATH}/{SOURCE_DATA_DIR}/"

if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 sync --quiet --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
else:
    cp = f"""
aws s3 sync --quiet --exclude .DS_Store {SOURCE_DATA_DIR} {S3_DEST}
aws s3 ls {S3_DEST}
"""
    
# execute the S3 copy
os.system(cp)

                           PRE 2014.04.21/
                           PRE 2014.04.22/
                           PRE 2014.04.23/
2024-03-01 20:40:58         57 sym


0

## Create Managed Database
Using the AWS APIs, create a managed database in Managed kdb Insights.

In [9]:
# assume it exists
create_db=False

try:
    resp = client.get_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)
    resp.pop('ResponseMetadata', None)
except:
    # does not exist, will create
    create_db=True

if create_db:
    print(f"CREATING Database: {DB_NAME}")
    resp = client.create_kx_database(environmentId=ENV_ID, databaseName=DB_NAME, description="Basictick kdb database")
    resp.pop('ResponseMetadata', None)

    print(f"CREATED Database: {DB_NAME}")

print(json.dumps(resp,sort_keys=True,indent=4,default=str))

CREATING Database: finspace-database
CREATED Database: finspace-database
{
    "createdTimestamp": "2024-03-01 20:40:58.628000+00:00",
    "databaseArn": "arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic/kxDatabase/finspace-database",
    "databaseName": "finspace-database",
    "description": "Basictick kdb database",
    "environmentId": "jlcenjvtkgzrdek2qqv7ic",
    "lastModifiedTimestamp": "2024-03-01 20:40:58.628000+00:00"
}


## Add HDB Data to Database
Add the data in the local hdb directory to the managed database using the changeset mechanism. The Data will be copied to S3 then ingested with the create-kx-changeset API.

In [10]:
changes=[]

for f in os.listdir(f"{SOURCE_DATA_DIR}"):
    if os.path.isdir(f"{SOURCE_DATA_DIR}/{f}"):
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}/", 'dbPath': f"/{f}/" } )
    else:
        changes.append( { 'changeType': 'PUT', 's3Path': f"{S3_DEST}{f}", 'dbPath': f"/" } )
        
resp = client.create_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, 
    changeRequests=changes)

resp.pop('ResponseMetadata', None)
changeset_id = resp['changesetId']

print("Changeset...")
print(json.dumps(resp,sort_keys=True,indent=4,default=str))

Changeset...
{
    "changeRequests": [
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.21/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.21/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.22/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.22/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/2014.04.23/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.23/"
        },
        {
            "changeType": "PUT",
            "dbPath": "/",
            "s3Path": "s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/sym"
        }
    ],
    "changesetId": "9sb939X2p1mpLa97bptOTg",
    "createdTimestamp": "2024-03-01 20:40:59.119000+00:00",
    "databaseName": "finspace-database",
    "environmentId":

In [11]:
wait_for_changeset_status(client, environmentId=ENV_ID, databaseName=DB_NAME, changesetId=changeset_id, show_wait=True)
print("**Done**")

Status is IN_PROGRESS, total wait 0:00:00, waiting 10 sec ...
**Done**


In [12]:
note_str = ""

c_set_list = list_kx_changesets(client, environmentId=ENV_ID, databaseName=DB_NAME)

if len(c_set_list) == 0:
    note_str = "<<Could not get changesets>>"
    
print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"  Changeset: {c_set_id}: Created: {c['createdTimestamp']} ({c['status']})")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: finspace-database, Changesets: 1 
  Changeset: 9sb939X2p1mpLa97bptOTg: Created: 2024-03-01 20:40:59.119000+00:00 (COMPLETED)


changeType,s3Path,dbPath
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.21/,/2014.04.21/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.22/,/2014.04.22/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/2014.04.23/,/2014.04.23/
PUT,s3://kdb-demo-829845998889-kms/data/TorQ-Amazon-FinSpace-Starter-Pack/hdb/sym,/


# Create Scaling Group
The scaling group represents the total compute avilable to the application. All clusters will be placed into the scaling group ans share the compute and memory of the scaling group.

In [13]:
resp = client.create_kx_scaling_group(
    environmentId = ENV_ID, 
    scalingGroupName = SCALING_GROUP_NAME,
    hostType=NODE_TYPE,
    availabilityZoneId = AZ_ID
)

In [14]:
resp

{'ResponseMetadata': {'RequestId': 'b2173643-c051-4523-8210-273cc930cb07',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '238',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 20:41:13 GMT',
   'x-amzn-requestid': 'b2173643-c051-4523-8210-273cc930cb07',
   'x-amz-apigw-id': 'T96IHFE9IAMEDpQ=',
   'x-amzn-trace-id': 'Root=1-65e23d66-12aa9d7426890a6b3ac887ca',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': 'BThjfUsT4JOkuKzoopRon8SEWJ-d3U5DSVQxXwlqqWTUNnH0xBABfw=='},
  'RetryAttempts': 0},
 'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'scalingGroupName': 'SCALING_GROUP_torq',
 'hostType': 'kx.sg.4xlarge',
 'availabilityZoneId': 'use1-az6',
 'status': 'CREATING',
 'lastModifiedTimestamp': datetime.datetime(2024, 3, 1, 20, 41, 12, 572000, tzinfo=tzlocal()),
 'createdTimestamp': datetime.datetime(2024, 3, 1, 20,

# Create Shared Volume
The shared volume is a common storage device for the application. Every cluster using the shared volume will have a writable directory named after the cluster, can read the directories named after other clusters in the application using the volume. Also, there is a common 

In [15]:
resp = client.create_kx_volume(
    environmentId = ENV_ID, 
    volumeType = 'NAS_1',
    volumeName = VOLUME_NAME,
    description = 'Shared volume between TP and RDB',
    nas1Configuration = NAS1_CONFIG,
    azMode='SINGLE',
    availabilityZoneIds=[ AZ_ID ]    
)

In [16]:
resp

{'ResponseMetadata': {'RequestId': 'c66b4ed3-5c1b-41d0-9202-a0dbcfc9fb05',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '431',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 20:41:14 GMT',
   'x-amzn-requestid': 'c66b4ed3-5c1b-41d0-9202-a0dbcfc9fb05',
   'x-amz-apigw-id': 'T96IfFScIAMEYlw=',
   'x-amzn-trace-id': 'Root=1-65e23d69-26856e8e31f80737480293f3',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': 'dvAlhFajzKW9f58ointAKgpTdNGcZDVxRYDaa25sHIHpGryPEs2fzA=='},
  'RetryAttempts': 0},
 'environmentId': 'jlcenjvtkgzrdek2qqv7ic',
 'volumeName': 'SHARED_torq',
 'volumeType': 'NAS_1',
 'volumeArn': 'arn:aws:finspace:us-east-1:829845998889:kxEnvironment/jlcenjvtkgzrdek2qqv7ic/kxVolume/SHARED_torq',
 'nas1Configuration': {'type': 'SSD_250', 'size': 1200},
 'status': 'CREATING',
 'azMode': 'SINGLE',
 'descript

# Wait for Volume and Scaling Group
Before proceeding to use Volumes and Scaling groups, wait for their creation to complete.

Volume will be used by the dataview.    
Dataview and Scaling Group will be used by the clusters


In [17]:
# wait for the scaling group to create
wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName=SCALING_GROUP_NAME, show_wait=True)
print("** DONE **")

# wait for the volume to create
wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, show_wait=True)
print("** DONE **")

Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:00:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:00:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:01:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:01:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:02:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:02:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:03:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:03:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:04:00, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is CREATING, total wait 0:04:30, waiting 30 sec ...
Scaling Group: SCALING_GROUP_torq status is now ACTIVE, total wait 0:0

# Create Dataview
Create a dataview, for a specific (static) version of the database and have all of its data cached using the shared volume.

In [18]:
# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

resp = client.create_kx_dataview(
    environmentId = ENV_ID, 
    databaseName=DB_NAME, 
    dataviewName=DBVIEW_NAME,
    azMode='SINGLE',
    availabilityZoneId=AZ_ID,
    changesetId=c_set_list[-1]['changesetId'],
    segmentConfigurations=[
        { 
            'dbPaths': ['/*'],
            'volumeName': VOLUME_NAME
        }
    ],
    autoUpdate=False,
    description = f'Dataview of database'
)

In [19]:
# wait for the view to create
wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, show_wait=True)
print("** DONE **")

Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:00:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:00:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:01:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:01:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:02:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:02:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:03:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:03:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:04:00, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total wait 0:04:30, waiting 30 sec ...
Dataview: finspace-database_DBVIEW status is CREATING, total

# Create Clusters
With foundation resources now completed, create the needed clusters for the application.

## Stage Code to S3
Code to be used in this application must be staged to an S3 bucket the service can read from, that code will then be deployed to the clusters as part of their creation workflow.

In [20]:
# zip the code
os.system(f"zip -q -r {CODEBASE}.zip {TORQ_CODEBASE}/ {TORQ_FINSPACE_CODEBASE}/ -x '*.ipynb_checkpoints*' -x '*/hdb/*' -x '*.git*' -x '*/tests/*' -x '*/terraform-deployment/*' -x '*/docs/*' -x '*/lib/*' -x '*/html/*' -x '*/datadog/*'  -x '*/monit/*'")

# copy code to S3
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

aws s3 cp --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
else:
    cp = f"""
aws s3 cp --exclude .DS_Store {CODEBASE}.zip s3://{S3_BUCKET}/code/{CODEBASE}.zip
aws s3 ls s3://{S3_BUCKET}/code/
"""
    
# execute the S3 copy
os.system(cp)

upload: ./torq_app.zip to s3://kdb-demo-829845998889-kms/code/torq_app.zip
2023-06-05 21:25:21          0 
2024-02-13 18:28:58      13775 basictick.zip
2024-02-16 21:40:43        542 code.zip
2023-12-21 19:47:37        574 codebundle.zip
2024-02-02 21:34:56        582 codebundle1.zip
2023-12-21 21:26:00        582 codebundle2.zip
2023-11-22 14:58:53       1530 jpmc_code.zip
2024-01-01 19:57:08      33781 kdb-tick-flat-largetable.zip
2023-12-30 22:56:33      38867 kdb-tick-flat.zip
2024-01-08 13:05:33      28741 kdb-tick.zip
2023-08-22 16:58:18        765 qcode.zip
2023-10-23 18:38:07       1390 taqcode.zip
2024-03-01 20:54:56     483538 torq_app.zip
2024-01-30 16:52:19       3583 tradeplus.zip


0

In [21]:
for c in clusters:
    
    # wait for a cluster?
    if c['type'] == "WAIT":
        wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=c['name'], show_wait=True)
        continue
    
    cluster_name = c['name']
    cluster_type = c['type']
    cluster_init = c['init']
    cluster_args = c['args']
    
    # cluster already exists
    resp = get_kx_cluster(client, environmentId=ENV_ID, clusterName=cluster_name)
    if resp is not None:
        print(f"Cluster: {cluster_name} already exists")
        continue
    
    print(f"Creating: {cluster_name}")
    
    resp = client.create_kx_cluster(
        environmentId=ENV_ID, 
        clusterName=cluster_name,
        clusterType=cluster_type,
        releaseLabel = '1.0',
        executionRole=EXECUTION_ROLE,
        databases=DATABASE_CONFIG,
        scalingGroupConfiguration={
            'memoryReservation': 6,
            'nodeCount': 1,
            'scalingGroupName': SCALING_GROUP_NAME,
        },
        savedownStorageConfiguration ={ 'volumeName': VOLUME_NAME },
#        tickerplantLogConfiguration ={ 'tickerplantLogVolumes': [ VOLUME_NAME ] },
        clusterDescription="Created with create_all notebook",
        code=CODE_CONFIG,
        initializationScript=cluster_init,
        commandLineArguments=cluster_args,
        azMode=AZ_MODE,
        availabilityZoneId=AZ_ID,
        vpcConfiguration={ 
            'vpcId': VPC_ID,
            'securityGroupIds': SECURITY_GROUPS,
            'subnetIds': SUBNET_IDS,
            'ipAddressType': 'IP_V4' }
    )
    
    display(resp)

Creating: discovery


{'ResponseMetadata': {'RequestId': 'f61c5e7b-7206-4bca-aeda-61f5c80f4391',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1672',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 20:54:59 GMT',
   'x-amzn-requestid': 'f61c5e7b-7206-4bca-aeda-61f5c80f4391',
   'x-amz-apigw-id': 'T98JDFUboAMEchQ=',
   'x-amzn-trace-id': 'Root=1-65e240a0-1ff460c97560ea35176ee0b7',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': '_uWpHfBtO6bxJ5MKHEEpGw-PETJN0738qaVZ1QDnjncvmgzhPTmLpA=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'discovery',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': '

Creating: rdb


{'ResponseMetadata': {'RequestId': 'e6148851-ea0b-4da2-b620-cfc5a93af3da',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1655',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 20:55:02 GMT',
   'x-amzn-requestid': 'e6148851-ea0b-4da2-b620-cfc5a93af3da',
   'x-amz-apigw-id': 'T98JqE-8oAMEGVQ=',
   'x-amzn-trace-id': 'Root=1-65e240a3-097124983f0126f408b89fb9',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': 'lj2jd_Tr7f_LfOCNULtMpl7-ITnzm_z33bVx9dyzgS1SMH1BpD6MAg=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'rdb',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'Wsb942

Creating: hdb


{'ResponseMetadata': {'RequestId': 'd9817bce-4290-4a9a-90e0-742da241d031',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1655',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 20:55:05 GMT',
   'x-amzn-requestid': 'd9817bce-4290-4a9a-90e0-742da241d031',
   'x-amz-apigw-id': 'T98KHG2WoAMEGRw=',
   'x-amzn-trace-id': 'Root=1-65e240a6-5d0c1076518493f904efad68',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': '3z8Mq4Fz87l0KhLAvXjJ0x4hD5ENBbTqx6PooKqGfU80rKyIAt2UPw=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'hdb',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'Wsb942

Cluster: discovery status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:05:30, waiting 30 sec ...
Cluster: discovery status is CREATING, total wait 0:06:00, waitin

{'ResponseMetadata': {'RequestId': 'f687caeb-446e-4648-b6f3-06b5a99f1e2b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1667',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 21:11:17 GMT',
   'x-amzn-requestid': 'f687caeb-446e-4648-b6f3-06b5a99f1e2b',
   'x-amz-apigw-id': 'T9-h4GWWIAMEbzA=',
   'x-amzn-trace-id': 'Root=1-65e24472-2a69d5d52351904227ec37ba',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': 'mJ5HpEYmVBe91MhEhETJbvt3w44B_QaVOzX2FJuPfLO6M1jvvT6PHg=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'gateway',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'Ws

Creating: feed


{'ResponseMetadata': {'RequestId': '0385423e-58da-47ab-bb99-27274c05a3de',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1669',
   'connection': 'keep-alive',
   'date': 'Fri, 01 Mar 2024 21:11:20 GMT',
   'x-amzn-requestid': '0385423e-58da-47ab-bb99-27274c05a3de',
   'x-amz-apigw-id': 'T9-icFLXoAMEnXg=',
   'x-amzn-trace-id': 'Root=1-65e24475-1501cb68413705f977c81a23',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 2b74e5ee4d30afba8f9df9907896c5f4.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD50-C2',
   'x-amz-cf-id': 'DmJRlxP2ZaXzfnzzcriVgHfjDTJf4x6TR9Ia7ZOFW4R51ucNJHXb-w=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'feed',
 'clusterType': 'GP',
 'volumes': [{'volumeName': 'SHARED_torq', 'volumeType': 'NAS_1'}],
 'databases': [{'databaseName': 'finspace-database',
   'cacheConfigurations': [],
   'dataviewConfiguration': {'dataviewName': 'finspace-database_DBVIEW',
    'dataviewVersionId': 'Wsb94

## Wait for all clusters to finish creating

In [22]:
# Wait for all clusters to start
for c in clusters:
    cluster_name = c['name']
    wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=cluster_name, show_wait=True)

print("** ALL DONE **")

Cluster: discovery status is now RUNNING, total wait 0:00:00
Cluster: rdb status is now RUNNING, total wait 0:00:00
Cluster: hdb status is now RUNNING, total wait 0:00:00
Cluster: discovery status is now RUNNING, total wait 0:00:00
Cluster: rdb status is now RUNNING, total wait 0:00:00
Cluster: gateway status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: gateway status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: gateway status is CRE

# List Clusters

In [23]:
cdf = get_clusters(client, environmentId=ENV_ID)

all_clusters = [d['name'] for d in clusters if 'name' in d]

if cdf is not None:
    cdf = cdf[cdf['clusterName'].isin(all_clusters)]

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
1,discovery,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'discovery'}, {'key': 'procname', 'value': 'discovery1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:10:34.715000+00:00,2024-03-01 20:54:59.373000+00:00,finspace-database,NaN
2,feed,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'tradeFeed'}, {'key': 'procname', 'value': 'tradeFeed1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:24:50.829000+00:00,2024-03-01 21:11:20.369000+00:00,finspace-database,NaN
3,gateway,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'gateway'}, {'key': 'procname', 'value': 'gateway1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:24:49.462000+00:00,2024-03-01 21:11:17.301000+00:00,finspace-database,NaN
4,hdb,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'hdb'}, {'key': 'procname', 'value': 'hdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '4'}]",Created with create_all notebook,2024-03-01 21:10:44.925000+00:00,2024-03-01 20:55:05.460000+00:00,finspace-database,NaN
5,rdb,RUNNING,GP,None,"[{'key': 'proctype', 'value': 'rdb'}, {'key': 'procname', 'value': 'rdb1'}, {'key': 'jsonlogs', 'value': 'true'}, {'key': 'noredirect', 'value': 'true'}, {'key': 's', 'value': '2'}]",Created with create_all notebook,2024-03-01 21:10:44.563000+00:00,2024-03-01 20:55:02.410000+00:00,finspace-database,NaN


# All Processes Running

In [24]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-03-01 21:25:00.567560
